In [ ]:
import streamlit as st
import asyncio
import io
import sys
from contextlib import redirect_stdout
from your_finance_agent_module import run_finance_agent

st.set_page_config(layout="wide", page_title="Finance Agent")

st.title("Finance Agent Query Interface")

# Sidebar for query input
with st.sidebar:
    st.header("Query Input")
    query = st.text_area("Enter your finance-related query:", height=100)
    run_button = st.button("Run Query", type="primary")

# Main content area
if run_button and query:
    # Create a placeholder for live updates
    progress_placeholder = st.empty()

    # Capture print statements
    output_capture = io.StringIO()
    
    async def run_and_capture():
        with redirect_stdout(output_capture):
            result = await run_finance_agent(query)
        return result

    with st.spinner("Processing your query..."):
        result = asyncio.run(run_and_capture())
    
    # Display captured output
    st.subheader("Process Log")
    with st.expander("View process details", expanded=True):
        st.code(output_capture.getvalue())
    
    # Display the result
    st.success("Query processing completed!")
    
    # Display graph database results if available
    if 'graph_db_result' in result:
        st.subheader("Graph Database Results")
        st.json(result['graph_db_result'])
    
    # Display expanded queries if available
    if 'expanded_queries' in result:
        st.subheader("Expanded Queries")
        for i, eq in enumerate(result['expanded_queries'], 1):
            st.write(f"{i}. {eq}")
    
    # Display subgraph results if available
    if 'subgraph_results' in result:
        st.subheader("Subgraph Results")
        for subgraph, subresult in result['subgraph_results'].items():
            with st.expander(f"Results from {subgraph}"):
                st.json(subresult)
    
    # Display final response
    st.subheader("Final Response")
    st.write(result.get('final_response', 'No final response available.'))

else:
    st.info("Enter a query in the sidebar and click 'Run Query' to start processing.")

# Footer
st.markdown("---")
st.markdown("© 2023 Finance Agent. All rights reserved.")